## Photometric error stage demo

author: Tianqing Zhang, John-Franklin Crenshaw

This notebook demonstrate the use of `rail.creation.degraders.photometric_errors`, which adds column for the  photometric noise to the catalog based on the package PhotErr developed by John-Franklin Crenshaw. The RAIL stage PhotoErrorModel inherit from the Noisifier base classes, and the LSST, Roman, Euclid child classes inherit from the PhotoErrorModel

In [ ]:

from rail.creation.degraders.photometric_errors import LSSTErrorModel
from rail.creation.degraders.photometric_errors import RomanErrorModel
from rail.creation.degraders.photometric_errors import EuclidErrorModel

from rail.core.data import PqHandle
from rail.core.stage import RailStage

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np



In [ ]:
DS = RailStage.data_store
DS.__class__.allow_overwrite = True


### Create a random catalog with ugrizy+YJHF bands as the the true input

In [ ]:
data = np.random.normal(23, 3, size = (1000,10))

data_df = pd.DataFrame(data=data,    # values
            columns=['u', 'g', 'r', 'i', 'z', 'y', 'Y', 'J', 'H', 'F'])
data_truth = PqHandle('input')
data_truth.set_data(data_df)

In [ ]:
data_df

### The LSST error model adds noise to the optical bands

In [ ]:
errorModel_lsst = LSSTErrorModel.make_stage(name="error_model")

samples_w_errs = errorModel_lsst(data_truth)
samples_w_errs()


In [ ]:
fig, ax = plt.subplots(figsize=(5, 4), dpi=100)

for band in "ugrizy":
    # pull out the magnitudes and errors
    mags = samples_w_errs.data[band].to_numpy()
    errs = samples_w_errs.data[band + "_err"].to_numpy()

    # sort them by magnitude
    mags, errs = mags[mags.argsort()], errs[mags.argsort()]

    # plot errs vs mags
    ax.plot(mags, errs, label=band)

ax.legend()
ax.set(xlabel="Magnitude (AB)", ylabel="Error (mags)")
plt.show()


### The Roman error model adds noise to the infrared bands

In [ ]:
errorModel_Roman = RomanErrorModel.make_stage(name="error_model", )



In [ ]:
errorModel_Roman.config['m5']['Y'] = 27.0

In [ ]:
errorModel_Roman.config['theta']['Y'] = 27.0

In [ ]:
samples_w_errs_roman = errorModel_Roman(data_truth)
samples_w_errs_roman()

In [ ]:
fig, ax = plt.subplots(figsize=(5, 4), dpi=100)

for band in "YJHF":
    # pull out the magnitudes and errors
    mags = samples_w_errs_roman.data[band].to_numpy()
    errs = samples_w_errs_roman.data[band + "_err"].to_numpy()

    # sort them by magnitude
    mags, errs = mags[mags.argsort()], errs[mags.argsort()]

    # plot errs vs mags
    ax.plot(mags, errs, label=band)

ax.legend()
ax.set(xlabel="Magnitude (AB)", ylabel="Error (mags)")
plt.show()


### The Euclid error model adds noise to YJH bands

In [ ]:
errorModel_Euclid = EuclidErrorModel.make_stage(name="error_model")

samples_w_errs_Euclid = errorModel_Euclid(data_truth)
samples_w_errs_Euclid()

In [ ]:
fig, ax = plt.subplots(figsize=(5, 4), dpi=100)

for band in "YJH":
    # pull out the magnitudes and errors
    mags = samples_w_errs_Euclid.data[band].to_numpy()
    errs = samples_w_errs_Euclid.data[band + "_err"].to_numpy()

    # sort them by magnitude
    mags, errs = mags[mags.argsort()], errs[mags.argsort()]

    # plot errs vs mags
    ax.plot(mags, errs, label=band)

ax.legend()
ax.set(xlabel="Magnitude (AB)", ylabel="Error (mags)")
plt.show()
